In [1]:
!pip install beautifulsoup4
!pip install lxml
!pip install html5lib
!pip install requests

In [40]:
import pandas as pd
import numpy as np
import requests

In [3]:
table = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
print(table[0])

    Postcode           Borough          Neighbourhood
0        M1A      Not assigned           Not assigned
1        M2A      Not assigned           Not assigned
2        M3A        North York              Parkwoods
3        M4A        North York       Victoria Village
4        M5A  Downtown Toronto           Harbourfront
..       ...               ...                    ...
282      M8Z         Etobicoke              Mimico NW
283      M8Z         Etobicoke     The Queensway West
284      M8Z         Etobicoke  Royal York South West
285      M8Z         Etobicoke         South of Bloor
286      M9Z      Not assigned           Not assigned

[287 rows x 3 columns]


In [4]:
data = pd.DataFrame(table[0])
data.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Not assigned


In [5]:
df = data.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(list)
df = df.sample(frac=1).reset_index()
df['Neighbourhood'] = df['Neighbourhood'].str.join(', ')
df

,Postcode,Borough,Neighbourhood
0,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
1,M4A,North York,Victoria Village
2,M1N,Scarborough,"Birch Cliff, Cliffside West"
3,M4Y,Downtown Toronto,Church and Wellesley
4,M4H,East York,Thorncliffe Park
...,...,...,...
175,M1R,Scarborough,"Maryvale, Wexford"
176,M1T,Scarborough,"Clarks Corners, Sullivan, Tam O'Shanter"
177,M2H,North York,Hillcrest Village
178,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ..."


In [6]:
df=df[df['Borough']!='Not assigned']
df[df['Neighbourhood']=='Not assigned']=df['Borough']
df

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:3503: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.loc._setitem_with_indexer(indexer, value)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:3484: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-d

,Postcode,Borough,Neighbourhood
0,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
1,M4A,North York,Victoria Village
2,M1N,Scarborough,"Birch Cliff, Cliffside West"
3,M4Y,Downtown Toronto,Church and Wellesley
4,M4H,East York,Thorncliffe Park
...,...,...,...
175,M1R,Scarborough,"Maryvale, Wexford"
176,M1T,Scarborough,"Clarks Corners, Sullivan, Tam O'Shanter"
177,M2H,North York,Hillcrest Village
178,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ..."


In [7]:
df.shape

(103, 3)

In [12]:
import sys
!{sys.executable} -m pip install geocoder

print('Packages installed')

Packages installed


In [20]:
coord = pd.read_csv("https://cocl.us/Geospatial_data")
coord.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [21]:
coord.columns

Index(['Postal Code', 'Latitude', 'Longitude'], dtype='object')

In [24]:
coord.rename(columns = {'Postal Code':'Postcode'},inplace=True)
coord.columns

Index(['Postcode', 'Latitude', 'Longitude'], dtype='object')

In [32]:
total = df.merge(coord,on='Postcode',how='left')
total.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
3,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
4,M4H,East York,Thorncliffe Park,43.705369,-79.349372
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6C,York,Humewood-Cedarvale,43.693781,-79.428191
7,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
8,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678
9,M6B,North York,Glencairn,43.709577,-79.445073


In [34]:
total.shape

(103, 5)

In [41]:
VERSION = '20180605' # Foursquare API version

name = total.loc[0, 'Neighbourhood'] # neighborhood name
latitude = total.loc[0, 'Latitude'] # neighborhood latitude value
longitude = total.loc[0, 'Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(name, 
                                                               latitude, 
                                                               longitude))
radius = 500 # define radius
LIMIT = 100 # limit of number of venues returned by Foursquare API

url = 'https://api.foursquare.com/v2/venues/explore?&client_id=JGGBRN5XODTLZGJOMCSWIQMRH1JLGJKPSFR10XNB2R5U25GR&client_secret=KWRAMLK2HOJBQ2XLICLKXRU3M4HOCC1U2VG4Y4OPP5JF03QX&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

results = requests.get(url).json()
results

Latitude and longitude values of Business Reply Mail Processing Centre 969 Eastern are 43.6627439, -79.321558.


{'meta': {'code': 200, 'requestId': '5e267011f7706a001c2061e9'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682094413784,
          'lng': -79.29394208780985}],
        'distanc

In [42]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [44]:
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Glen Stewart Ravine,Other Great Outdoors,43.676300,-79.294784
3,Grover Pub and Grub,Pub,43.679181,-79.297215
4,Upper Beaches,Neighborhood,43.680563,-79.292869
